<a href="https://colab.research.google.com/github/surabhi13gupta/LangChains/blob/main/chatprompt_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 3.9 MB/s eta 0:00:00


In [2]:
!pip install langchain_openai
!pip install langchain_core

In [3]:
from google.colab import userdata
import os
github_token = userdata.get('GITHUB_TOKEN')
os.environ['GITHUB_TOKEN'] = github_token

In [4]:
from langchain_openai import ChatOpenAI

In [8]:
endpoint = "https://models.github.ai/inference"
model_name = "openai/gpt-4o-mini"

In [9]:
model = ChatOpenAI(
    base_url = endpoint,
    api_key= github_token,
    model=model_name,
    max_completion_tokens=100
)

In [10]:
while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        break
    result = model.invoke(user_input)
    print("AI: ", result.content)

You: Hi
AI:  Hello! How can I assist you today?
You: Which one is greater 2 or 5?
AI:  5 is greater than 2.
You: multiply greater number by 10
AI:  To multiply the greater number by 10, you need to identify which of the two numbers is greater. Please provide two numbers, and I'll help you multiply the greater one by 10.
You: exit


## problem with above : chatbot does not have the previous chat history and no context retention

In [12]:
chat_history = []
while True:
    user_input = input("You: ")
    chat_history.append(user_input)
    if user_input.lower() == 'exit':
        break
    result = model.invoke(chat_history)
    chat_history.append(result.content)
    print("AI: ", result.content)

print("Chat History")

print(chat_history)

You: Hi
AI:  Hello! How can I assist you today?
You: which one is greater 5 or 2?
AI:  5 is greater than 2.
You: now multiply greater by 10
AI:  Multiplying 5 by 10 gives you 50.
You: exit
Chat History
['Hi', 'Hello! How can I assist you today?', 'which one is greater 5 or 2?', '5 is greater than 2.', 'now multiply greater by 10', 'Multiplying 5 by 10 gives you 50.', 'exit']


Problem: Chat history does not show who sends what. - no role information

Langchain identified this issue already and resolved also

## Now lets use different types of messages provided by LangChain

In [13]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

In [14]:
messages = [
    SystemMessage(content="You are a helpful assistant"),
    HumanMessage(content="Tell me about langchain"),
]
result = model.invoke(messages)
messages.append(AIMessage(content=result.content))
print(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about langchain', additional_kwargs={}, response_metadata={}), AIMessage(content='LangChain is a framework designed for developing applications powered by language models. It offers tools and features that simplify creating interactive and sophisticated applications that leverage natural language processing (NLP) capabilities. Key components of LangChain include:\n\n1. **Modular Components**: LangChain provides a modular approach to building applications. This includes various components such as language models, chains, prompts, and agents that can be easily combined and customized to fit specific use cases.\n\n2. **Chains**: Chains are a core', additional_kwargs={}, response_metadata={})]


In [15]:
messages

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Tell me about langchain', additional_kwargs={}, response_metadata={}),
 AIMessage(content='LangChain is a framework designed for developing applications powered by language models. It offers tools and features that simplify creating interactive and sophisticated applications that leverage natural language processing (NLP) capabilities. Key components of LangChain include:\n\n1. **Modular Components**: LangChain provides a modular approach to building applications. This includes various components such as language models, chains, prompts, and agents that can be easily combined and customized to fit specific use cases.\n\n2. **Chains**: Chains are a core', additional_kwargs={}, response_metadata={})]

## Redefine chat bot using these message types

In [16]:
chat_history = [SystemMessage(content="You are a helpful assistant!")]
while True:
    user_input = input("You: ")
    chat_history.append(HumanMessage(content = user_input))
    if user_input.lower() == 'exit':
        break
    result = model.invoke(chat_history)
    chat_history.append(AIMessage(content = result.content))
    print("AI: ", result.content)

print("Chat History")

print(chat_history)

You: Hi
AI:  Hello! How can I assist you today?
You: Which one is greater 5 or 2
AI:  5 is greater than 2.
You: Multiply greater by 10
AI:  Multiplying the greater number, 5, by 10 gives you:

\( 5 \times 10 = 50 \) 

So, the result is 50.
You: exit
Chat History
[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='Which one is greater 5 or 2', additional_kwargs={}, response_metadata={}), AIMessage(content='5 is greater than 2.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Multiply greater by 10', additional_kwargs={}, response_metadata={}), AIMessage(content='Multiplying the greater number, 5, by 10 gives you:\n\n\\( 5 \\times 10 = 50 \\) \n\nSo, the result is 50.', additional_kwargs={}, response_metadata={}), HumanMessage(co

In [17]:
chat_history

[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Which one is greater 5 or 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='5 is greater than 2.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Multiply greater by 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Multiplying the greater number, 5, by 10 gives you:\n\n\\( 5 \\times 10 = 50 \\) \n\nSo, the result is 50.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='exit', additional_kwargs={}, response_metadata={})]

## Now use ChatPromptTemplate which comes with more benefits - dynamic message creation using list of messages

In [18]:
from langchain_core.prompts import ChatPromptTemplate

In [22]:
chat_prompt_template = ChatPromptTemplate([
    ('system', 'You are a helpful {domain} expert'),
    ('human', 'Explain in simple terms, what is {topic}')
])
prompt = chat_prompt_template.invoke({
    "domain": "cricket",
    "topic": "run-out"
})
prompt

ChatPromptValue(messages=[SystemMessage(content='You are a helpful cricket expert', additional_kwargs={}, response_metadata={}), HumanMessage(content='Explain in simple terms, what is run-out', additional_kwargs={}, response_metadata={})])

#Message Placeholder

insert chat history or a list of messages at runtime

In [23]:
from langchain_core.prompts import MessagesPlaceholder

In [26]:
chat_template = ChatPromptTemplate([
    ('system', 'You are a helpful customer support agent'),
    MessagesPlaceholder(variable_name = 'chat_history'),
    ('human', '{query}')
]
)

#load chat history from server or database
#Invoke model putting chat_history and query